<a href="https://colab.research.google.com/github/purvi202211023/IT555-Deep-Neural-NLP/blob/Train-Word2Vec-on-peS2o-Dataset-(AllenNLP)-_Assignment1/Deep_Neural_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install num2words
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import inflect
import pandas as pd
import numpy as np
from num2words import num2words  # For numerical-to-text conversion

In [ ]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# # Load the training data
# df_train_n = pd.read_pickle("/content/drive/MyDrive/NLP/train_data_n.pkl")

# # Load the test data
# df_test_n = pd.read_pickle("/content/drive/MyDrive/NLP/test_data_n.pkl")

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/df_new.pkl")
df_train = df[:3000]
df_test = df[3000:4000]

In [ ]:
df_train

,id,source,version,added,created,text
0,79374620,s2ag/train,v1,2019-03-16T13:05:58.356Z,2006-01-01T00:00:00.000Z,In-time rating of continuously cast semis by m...
1,53596570,s2ag/train,v1,2018-11-07T20:43:16.101Z,2003-09-24T00:00:00.000Z,Development of an Inspection System for Cracks...
2,60680010,s2ag/train,v1,2019-02-13T14:02:18.109Z,2004-01-01T00:00:00.000Z,Assessment of Wave-Induced Liquefaction in a P...
3,254100620,s2ag/train,v1,2022-12-01T14:30:17.430Z,2022-10-10T00:00:00.000Z,Prediction of Cascading Failures and Simultane...
4,17912070,s2ag/train,v1,2014-10-01T00:00:00.000Z,2001-01-01T00:00:00.000Z,2 3 Ju l 2 00 1 A Logical Framework for Conver...
...,...,...,...,...,...,...
2995,69728870,s2ag/train,v1,2019-02-19T14:07:21.453Z,2018-06-25T00:00:00.000Z,Learning Experience Technology Usability Desig...
2996,63439480,s2ag/train,v1,2019-02-16T14:28:03.063Z,2009-08-31T00:00:00.000Z,Data Mining in Biomedicine Using Ontologies\n\...
2997,251327930,s2ag/train,v1,2022-08-05T10:28:00.914Z,2020-01-01T00:00:00.000Z,The Situation and Development Route of Greenho...
2998,10970270,s2ag/train,v1,2017-02-17T00:19:40.761Z,2015-09-02T00:00:00.000Z,Collaborative Composition of Production Servic...


In [ ]:
# Create an inflect engine for number-to-text conversion
p = inflect.engine()

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove bullets
    text = re.sub(r'\u2022', '', text)

    # Handle apostrophe
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)

    # Handle hyphens
    text = text.replace("-", " ")
    text = text.replace("_", " ")

    # Handle enumerations
    text = re.sub(r'\(i\)|\(ii\)|\(iii\)|\(iv\)|\(v\)|\(vi\)|\(vii\)|\(viii\)|\(ix\)|\(x\)', '', text)

    # Convert numbers to text
    text = ' '.join([p.number_to_words(word) if bool(re.search(r'\d', word)) else word for word in text.split()])

    # Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize and remove stopwords
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.lower() not in stopwords.words('english')]

    return ' '.join(tokens)

# Tokenize the text in df_train into sentences
sentences_train = [sent_tokenize(text) for text in df_train['text']]
sentences_train = [sentence for sublist in sentences_train for sentence in sublist]  # Flatten the list

# Apply the pre-processing function to each sentence
processed_sentences = [preprocess_text(sentence) for sentence in sentences_train]

processed_sentences[:5]  # Display first 5 processed sentences


['time rating continuously cast semis means hybrid quality models efficient reliable time determination surface internal quality continuous cast products deliver considerable contribution cost reduction efficiency improvement steel plants',
 'currently economic losses environmental impacts caused downgrading slabs billets additionally due conditioning increased material energy consumption occur',
 'nowadays development quality prediction models supported availability automatic inspection systems eg',
 'surface inspection hot cold rolled strips bars',
 'developing validating quality model concerning defects occur downstream process steps casting information inspection system reliable human decisions model adjusted accuracy']

In [ ]:
def check_for_numbers(processed_sentences):
    sentences_with_numbers = [sentence for sentence in processed_sentences if bool(re.search(r'\d', sentence))]
    return sentences_with_numbers

# Using the function
sentences_with_numbers = check_for_numbers(processed_sentences)

if sentences_with_numbers:
    print(f"Found {len(sentences_with_numbers)} sentences with numbers:")
    for sent in sentences_with_numbers:
        print(sent)
else:
    print("No numbers found in the processed sentences!")


No numbers found in the processed sentences!


In [ ]:
len(processed_sentences)

24432

In [ ]:
# # Function to apply all preprocessing steps
# def preprocess_text(text):
#     # Convert text to lowercase
#     text = text.lower()

#     # Remove URLs
#     text = re.sub(r'http\S+', '', text)

#     # Remove bullets and enumerations
#     text = re.sub(r'•|\d+\.', '', text)

#     # Remove apostrophes
#     text = re.sub(r"([’'‘’])", '', text)

#     # Remove hyphens
#     text = re.sub(r"-", ' ', text)

#     # Convert numerical digits to text
#     text = ' '.join([num2words(word) if word.isdigit() else word for word in text.split()])

#     # Tokenize the text
#     tokens = word_tokenize(text)

#     # Remove stopwords
#     stop_words = set(stopwords.words('english'))
#     tokens = [word for word in tokens if word.lower() not in stop_words]

#     # Remove punctuation
#     tokens = [word for word in tokens if word.isalnum()]

#     # Join tokens back into a single string
#     text = ' '.join(tokens)

#     return text

# # Apply the preprocessing function to the "text" column
# combined_df['text'] = combined_df['text'].apply(preprocess_text)

# # Print the preprocessed DataFrame
# print(combined_df)

In [ ]:
# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit the vectorizer to the training data and transform it to create the context matrix
context_matrix_train = tfidf_vectorizer.fit_transform(processed_sentences)


In [ ]:
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

In [ ]:
# Convert the TF-IDF context matrix to a DataFrame for easier viewing
context_matrix_train_df = pd.DataFrame(context_matrix_train.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Display the DataFrame
print(context_matrix_train_df)

       aaai  aacc  aace  aachen  aacr  aade  aalto  aamas  aaohns  aarhus  \
0       0.0   0.0   0.0     0.0   0.0   0.0    0.0    0.0     0.0     0.0   
1       0.0   0.0   0.0     0.0   0.0   0.0    0.0    0.0     0.0     0.0   
2       0.0   0.0   0.0     0.0   0.0   0.0    0.0    0.0     0.0     0.0   
3       0.0   0.0   0.0     0.0   0.0   0.0    0.0    0.0     0.0     0.0   
4       0.0   0.0   0.0     0.0   0.0   0.0    0.0    0.0     0.0     0.0   
...     ...   ...   ...     ...   ...   ...    ...    ...     ...     ...   
24427   0.0   0.0   0.0     0.0   0.0   0.0    0.0    0.0     0.0     0.0   
24428   0.0   0.0   0.0     0.0   0.0   0.0    0.0    0.0     0.0     0.0   
24429   0.0   0.0   0.0     0.0   0.0   0.0    0.0    0.0     0.0     0.0   
24430   0.0   0.0   0.0     0.0   0.0   0.0    0.0    0.0     0.0     0.0   
24431   0.0   0.0   0.0     0.0   0.0   0.0    0.0    0.0     0.0     0.0   

       ...   이차  키워드   탐색  프로파일  프로파일의  ﬁeld  ﬁelds  ﬁnal  ﬁve  ﬂight  
0  

multiply matrix with hot vector

In [ ]:
from gensim.models import Word2Vec

# # Tokenize the sentences in your preprocessed training data
tokenized_sentences = [sentence.split() for sentence in processed_sentences]

# # Set Word2Vec hyperparameters
# vector_size = 100  # Adjust the vector size as needed
# window = 5  # Adjust the context window size as needed
# min_count = 1  # Minimum word count, adjust as needed
# sg = 0  # Use CBOW (Continuous Bag of Words) model, set to 1 for Skip-gram

# # Initialize and train the Word2Vec model
# word2vec_model = Word2Vec(tokenized_sentences, vector_size=vector_size, window=window, min_count=min_count, sg=sg)


In [ ]:
model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)
model.train(tokenized_sentences, total_examples=len(tokenized_sentences), epochs=100)

(37882357, 39367400)

In [ ]:
def tfidf_weighted_word2vec_optimized(tfidf_matrix, tfidf_feature_names, w2v_model):
    # Precompute word embeddings for all the vocabulary words
    word_embeddings = {word: w2v_model.wv[word] for word in tfidf_feature_names if word in w2v_model.wv}

    # Convert word embeddings to a matrix form
    embedding_matrix = np.zeros((len(tfidf_feature_names), w2v_model.vector_size))
    for i, word in enumerate(tfidf_feature_names):
        if word in word_embeddings:
            embedding_matrix[i] = word_embeddings[word]

    # Use matrix multiplication to get the document embeddings
    doc_embeddings = tfidf_matrix.dot(embedding_matrix)

    return doc_embeddings

In [ ]:
doc_embeddings = tfidf_weighted_word2vec_optimized(context_matrix_train, tfidf_feature_names, model)
print(doc_embeddings)

[[ 0.4678576   2.78063561  1.07777167 ...  0.43808938  0.88900405
   3.71124423]
 [ 0.46219418  3.00381085  3.25360596 ... -0.30098841 -1.93462738
   1.83743916]
 [ 1.71205679 -0.89931819 -3.04423984 ... -1.33029437  0.85944849
   0.53314669]
 ...
 [ 0.52439886 -1.04842381 -0.60738323 ... -1.64926373 -1.47665975
  -1.02158715]
 [ 0.32080643  4.14160527  0.14475278 ...  0.03515094  0.14039905
  -0.42089222]
 [-1.9811232  -0.02916519 -4.57580832 ... -2.39396451 -1.54370425
   2.28455439]]


In [ ]:
# Tokenize the text in df_train into sentences
sentences_test = [sent_tokenize(text) for text in df_test['text']]
sentences_test = [sentence for sublist in sentences_test for sentence in sublist]  # Flatten the list

# Apply the pre-processing function to each sentence
processed_sentences_test = [preprocess_text(sentence) for sentence in sentences_test]

processed_sentences_test[:5]  # Display first 5 processed sentences

['simulated annealing metaheuristic solve optimal power flow optimal power flow problem widely studied order improve power systems operation planning',
 'real power systems problem formulated non linear large combinatorial problem',
 'first approaches used solve problem based mathematical methods required huge computational efforts',
 'lately artificial intelligence techniques metaheuristics based biological processes adopted',
 'metaheuristics require lower computational resources clear advantage addressing problem large power systems']

In [ ]:
print(model.predict_output_word(['Computer', 'science', 'information', 'technology']))

[('computer', 0.9996924), ('department', 1.7939261e-05), ('engineering', 1.1264711e-05), ('technology', 8.486653e-06), ('communication', 6.089703e-06), ('communications', 5.221026e-06), ('information', 3.8731996e-06), ('age', 3.7926332e-06), ('universiti', 3.3899782e-06), ('educators', 2.6036362e-06)]


In [ ]:
print(model.predict_output_word(['What', 'is', 'Computer', 'science']))

[('computer', 1.0), ('mathematics', 4.325865e-13), ('stem', 2.2076714e-14), ('cognitive', 2.196654e-14), ('department', 1.11988445e-14), ('engineering', 1.0948807e-14), ('plasma', 2.4605852e-15), ('instruction', 1.2121784e-15), ('optometric', 6.3025683e-16), ('section', 6.2991013e-16)]


In [ ]:
# Initialize an empty list to store the predictions for each test sentence
test_predictions = []

# Iterate through each test sentence
for test_sentence in processed_sentences_test:
    # Filter out words that are not in the model's vocabulary
    test_sentence_in_vocab = [word for word in test_sentence if word in model.wv]

    # Check if there are any words left in the test sentence after filtering
    if test_sentence_in_vocab:
        # Use the model to predict the most similar sentences to the test sentence
        similar_sentences = model.wv.most_similar(positive=test_sentence_in_vocab, topn=5)
        test_predictions.append(similar_sentences)
    else:
        # If the test sentence is empty after filtering, append an empty list to test_predictions
        test_predictions.append([])

# Display the predictions for the first test sentence
print("Predictions for the first test sentence:")
print(processed_sentences_test[0])
for prediction in test_predictions[0]:
    print(prediction)


Predictions for the first test sentence:
simulated annealing metaheuristic solve optimal power flow optimal power flow problem widely studied order improve power systems operation planning
('joanna', 0.7350724339485168)
('rien', 0.7318426370620728)
('ommunity', 0.7275989055633545)
('trouillot', 0.7237244844436646)
('leong', 0.7222837805747986)


In [ ]:
# Step 3: Create TF-IDF-weighted Word2Vec representations
# def tfidf_weighted_word2vec(tfidf_matrix, tfidf_feature_names, w2v_model):
#     doc_embeddings = []

#     for doc_tfidf in tfidf_matrix:
#         doc_embedding = np.zeros(w2v_model.vector_size)
#         for word, weight in zip(tfidf_feature_names, doc_tfidf.toarray()[0]):
#             if word in w2v_model.wv:
#                 doc_embedding += weight * w2v_model.wv[word]
#         doc_embeddings.append(doc_embedding)

#     return np.array(doc_embeddings)

# doc_embeddings = tfidf_weighted_word2vec(context_matrix_train, tfidf_feature_names, model)
# print(doc_embeddings)